In [1]:
import gensim
from gensim.models import doc2vec
from gensim.models.doc2vec import TaggedDocument
from collections import namedtuple
import pandas as pd
import smart_open
import random

In [2]:
%matplotlib inline
from preamble import *
plt.rcParams['image.cmap'] = "gray"
import re

# 인터뷰 텍스트 데이터 마이닝

1. 데이터 전처리 과정
     - 인터뷰 text 데이터 불러오기
     - 형태소 분석을 통한 POS 태깅
     - docs_ko : 원문서
     - tokens_ko : 원문서를 형태소단위로 쪼갠 후 토큰화시키는 문서 (konlpy.tag 를 사용하여)
     - ko : Token 을 wapper 한다. (nlpy의 기능을 사용한다)
     - tag_text_ko : 토큰을 의미단위로 나눕니다 ( 형태소 분석기 사용)
2. 데이터 분석 과정
    - doc2vec 에서 요구하는 전처리를 한번 더해준다
    - 사전구축하고, vector들을 트레이닝시킨다.
    - Topic Modeling : 어떤 문서에서 자주 나타는 단어를 통해, 주제를 찾아주는 확률적인 모델을 디자인합니다.유명한 것으로 LDA, LSI, HDP가 있습니다.
    - Word Embedding : 문장속의 단어간의 관계를 비지도 학습 방식으로 분석하여 만들어지는 수십~ 수백차원의 벡터로서 특징(Feature)화 되는 단어들을 만들어 냅니다. 단어가 가지고있는 벡터간의 연산을 통해 다른 단어와의 관계를 만들어내게 됩니다
    

In [3]:
#인터뷰 text 데이터 불러오기 + 예외처리

from konlpy.corpus import kobill
docs_ko = []
for i in kobill.fileids():
    try:
        docs_ko.append(kobill.open(i).read())
    except UnicodeDecodeError:
        print(i)

In [4]:
# 12개 인터뷰 데이터
print(docs_ko)

['DSJY: \t자 저희는 유니스트 소속 대학원 수업 CONTEXTUAL DESGIN 을 수강하고 있는 이지영입니다. 저희는 학생창업자로서 겪는 여러가지 경험을 자세히 듣고자하며, 특히 업무 진행, 갈등 중재, 활용하는 테크놀로지에 대해 관심이 많아서 이렇게 인터뷰를 진행하게 되었습니다. 인터뷰는 약 30분정도 진행할 것 같고요. 그럼 시작 할꼐요.\nP2\t네\nDSJY\t보나는 어떤 집단인지 궁금합니다\nP2\t우리는 습관에 대해서 항상하고 사람들이 움직이는 방법에 대해서 건강한 습관을 가지고 내가 원하는것, 보나라는 이름이 원래 보고싶은 나라는 뜻이에요. 내가 원하는것과 하고싶어하는 것을 습관을 통해서 이루어간다는 비전을 가지고 있고요. 지금 하는 것은 여러가지 시도를 해보다가 현재는 건강음식 건강음료 이런것을 유니스트의 학생들이 건강하게 살면서 할 수 있도록 지금 준비중에 있습니다.\nDSJY\t굉장히 좋은 취지인것 같습니다\nP2\t그렇죠? 우리팀에서 일할래?\nDSJY\t돈주면..., 보나의 규모가 어느정도인가요?\nP2\t규모라는게 너무 작아서 규모라고 할 정도가 아니고 정말 작아서 그냥 1인기업, 스타트업이라고 생각하시면 될것 같습니다. 풀타임 워커가 3명, 하프타임이 1명, 풀타임이라는것은 9시부터 나와서 9시부터 5시반까지 지키는 사람들 보통은 그 시간 이상까지 있고요. 나머지는 수업이 있고 갔다오고 그러고요. 매출이나 그런게 없다보니 규모로 인원 규모 밖에 말씀을 못드립니다.\nDSJY\t업무환경에 대해서 말씀해 주실수 있나요? 오피스의 컴퓨터라던지?\nP\t2오피스에 컴퓨터 한대 그리고 회의가능하게 화이트보드가 있고, 업무환경의 소프트웨어같은 쪽에서는 잔디,슬랙 이런것 중에서 저희는 잔디를 주로 사용을 하고요. 카톡과는 다른 업무용 메신저를 써서 이제 엄무시간 동안에는 될수 있도록 카톡을 안들어갈려고 하는 중입니다. 주변에서 우리끼리도 잔디를 쓰고 카톡을 키면 일하지 않는 걸로. 매번 안할수는 없겠지만 켜두지 말자는 거죠. 컴퓨터로 일하

In [5]:
type(docs_ko)

list

In [6]:
#POS 테깅하기 : 형태소를 비롯하여, 어근, 접두사, 접미사, 품사(POS) 등 다양한 언어적 속성을 분석한다.
from konlpy.tag import Twitter; t = Twitter()
pos = lambda d: ['/'.join(p) for p in t.pos(d, stem=True, norm=True)]
tag_texts_ko = [pos(doc) for doc in docs_ko]
print(tag_texts_ko)

[['DSJY/Alpha', ':/Punctuation', '자/Noun', '저희/Noun', '는/Josa', '유니스트/Noun', '소속/Noun', '대학원/Noun', '수업/Noun', 'CONTEXTUAL/Alpha', 'DESGIN/Alpha', '을/Josa', '수강/Noun', '하고/Josa', '있다/Adjective', '이지영/Noun', '이다/Adjective', './Punctuation', '저희/Noun', '는/Josa', '학생/Noun', '창업/Noun', '자/Suffix', '로서/Noun', '겪다/Verb', '여러가지/Noun', '경험/Noun', '을/Josa', '자세하다/Adjective', '듣다/Verb', '자하/Noun', '며/Josa', ',/Punctuation', '특히/Adverb', '업무/Noun', '진행/Noun', ',/Punctuation', '갈등/Noun', '중재/Noun', ',/Punctuation', '활용/Noun', '하다/Verb', '테크놀로지/Noun', '에/Josa', '대해/Noun', '관심/Noun', '이/Josa', '많다/Adjective', '이렇게/Adverb', '인터뷰/Noun', '를/Josa', '진행/Noun', '하다/Verb', '되어다/Verb', './Punctuation', '인터뷰/Noun', '는/Josa', '약/Noun', '30/Number', '분/Noun', '정도/Noun', '진행/Noun', '하다/Verb', '것/Noun', '같다/Adjective', './Punctuation', '그렇다/Adjective', '시작/Noun', '하다/Verb', './Punctuation', 'P/Alpha', '2/Number', '네/Noun', 'DSJY/Alpha', '보나/Noun', '는/Josa', '어떻다/Adjective', '집단/Noun', '인지/Josa', '궁금하다/Adjective'

In [7]:
type(tag_texts_ko)

list

In [8]:
morph_cate = []
for item in tag_texts_ko[0]:
        b = item.split('/')
        c = b[1]
        morph_cate.append(c)

In [9]:
set(morph_cate)

{'Adjective',
 'Adverb',
 'Alpha',
 'Conjunction',
 'Determiner',
 'Exclamation',
 'Josa',
 'Noun',
 'NounPrefix',
 'Number',
 'Punctuation',
 'Suffix',
 'Verb',
 'VerbPrefix'}

In [10]:
remove_item = ('Conjunction', 'Determiner', 'Exclamation','Josa','Punctuation','Suffix')

In [11]:
filtered_list = []
for item in tag_texts_ko[0]:
    remove = False #전체 데이터
    for i in remove_item: 
        length = len(i)
        if item[-length:] == i:
            remove = True #삭제할 친구
    if not remove:
        filtered_list.append(item) # 삭제 안한 애들만 출력, False만 출력

In [12]:
data_set = [filtered_list]
print(data_set)

[['DSJY/Alpha', '자/Noun', '저희/Noun', '유니스트/Noun', '소속/Noun', '대학원/Noun', '수업/Noun', 'CONTEXTUAL/Alpha', 'DESGIN/Alpha', '수강/Noun', '있다/Adjective', '이지영/Noun', '이다/Adjective', '저희/Noun', '학생/Noun', '창업/Noun', '로서/Noun', '겪다/Verb', '여러가지/Noun', '경험/Noun', '자세하다/Adjective', '듣다/Verb', '자하/Noun', '특히/Adverb', '업무/Noun', '진행/Noun', '갈등/Noun', '중재/Noun', '활용/Noun', '하다/Verb', '테크놀로지/Noun', '대해/Noun', '관심/Noun', '많다/Adjective', '이렇게/Adverb', '인터뷰/Noun', '진행/Noun', '하다/Verb', '되어다/Verb', '인터뷰/Noun', '약/Noun', '30/Number', '분/Noun', '정도/Noun', '진행/Noun', '하다/Verb', '것/Noun', '같다/Adjective', '그렇다/Adjective', '시작/Noun', '하다/Verb', 'P/Alpha', '2/Number', '네/Noun', 'DSJY/Alpha', '보나/Noun', '어떻다/Adjective', '집단/Noun', '궁금하다/Adjective', 'P/Alpha', '2/Number', '우리/Noun', '습관/Noun', '대하다/Verb', '항상/Noun', '사람/Noun', '움직이다/Verb', '방법/Noun', '대하다/Verb', '건강하다/Adjective', '습관/Noun', '가지/Noun', '내/Noun', '원/Noun', '하다/Verb', '보나/Noun', '이름/Noun', '원래/Noun', '보다/Verb', '나라/Noun', '뜻/Noun', '내/Noun', '원/Noun

In [13]:
from gensim.models import doc2vec
from gensim.models.doc2vec import TaggedDocument
from collections import namedtuple
words = set(sum(data_set,[]))
print(words)

{'적절하다/Adjective', '인사이트/Noun', '공동체/Noun', '컴퓨터/Noun', '해/Noun', '잡다/Verb', '존재/Noun', '그러면/Adverb', '대외/Noun', '자주/Noun', '목표/Noun', '프론트/Noun', '걸다/Verb', '성공/Noun', '이기/Noun', '일어나다/Verb', '환경/Noun', '걸/Noun', '보나/Noun', '결국/Adverb', '대하다/Verb', '자기/Noun', '역활/Noun', '들이다/Verb', '출산/Noun', 'WELLNESS/Alpha', '끌/Noun', '봐/Noun', '움직이다/Verb', '작성/Noun', '편입/Noun', '배경/Noun', '아직/Noun', '조금/Noun', '알리다/Verb', '끝나다/Verb', '음/Noun', '와/Noun', '스케쥴/Noun', '완성/Noun', '세운/Noun', '이렇게/Adverb', '시뮬레이션/Noun', '특징/Noun', '이지영/Noun', '정/Noun', '커뮤니케이션/Noun', '친근하다/Adjective', '할껀/Noun', '이런툴/Noun', '뭐라다/Verb', '인터뷰/Noun', '다/Adverb', '글/Noun', '취지/Noun', '상황/Noun', '매번/Noun', '일별/Noun', '다른/Noun', '경험/Noun', '수업/Noun', '누리/Noun', '공식/Noun', 'P/Alpha', '신선하다/Adjective', '눕다/Verb', '빠르다/Adjective', '겁니다/Verb', '시반/Noun', '말/Noun', '기업가/Noun', '개발자/Noun', '그것/Noun', '달라지다/Verb', '책임/Noun', '더/Noun', '본인/Noun', '타이틀/Noun', '아니다/Adjective', '딱하다/Adjective', '자/Noun', '과정/Noun', '잘못/Noun', '마찰/Noun', 